In [1]:
### =============================================================================
# IMPORTS
# =============================================================================
import os
# Disable albumentations update warning
os.environ['NO_ALBUMENTATIONS_UPDATE'] = '1'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image
import cv2
from pathlib import Path
from typing import Dict, List, Tuple
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

# Import benchmarking and timing libraries
import time
import pandas as pd
import joblib # For saving/loading models

# Import specific ML model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler 

# Disable CUML (GPU) support
cuml_available = False
print("CUML/CUPY support is disabled. Running Sklearn models on CPU only.")

# =============================================================================
# CONFIGURATION
# =============================================================================
class Config:
    """Configuration class for data, models, and training parameters."""
    def __init__(self):
        # Basic configurations
        self.seed = 42
        
        # Paths
        # NOTE: Adjust these paths if necessary for your environment
        self.data_dir = Path("/kaggle/input/orange-dataset/Processed_data")
        self.output_dir = Path("/kaggle/working")
        
        # Categories
        self.categories = ['citrus canker', 'healthy', 'melanose']
        self.num_classes = len(self.categories)
        
        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def plot_confusion_matrix(y_true: List[int], y_pred: List[int], class_names: List[str], save_path: str, model_name: str):
    """Plots and saves the confusion matrix for a specific model."""
    plt.rcParams.update({'font.size': 18})
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 18})
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Confusion matrix for {model_name} saved to {save_path}")

def prepare_data(config: Config) -> Tuple[List, List]:
    """Prepare data paths and labels."""
    images, labels = [], []
    label_dict = {cat: idx for idx, cat in enumerate(config.categories)}
    
    print("Loading data paths...")
    for category in config.categories:
        category_path = config.data_dir / category
        image_paths = list(category_path.glob('*.jpg'))
        print(f"{category}: {len(image_paths)} images")
        images.extend(image_paths)
        labels.extend([label_dict[category]] * len(image_paths))
    
    return images, labels

# =============================================================================
# TRADITIONAL FEATURE EXTRACTION
# =============================================================================

def get_lbp_features(image: np.ndarray, num_points: int = 24, radius: int = 8) -> np.ndarray:
    """Calculates Local Binary Patterns (LBP) features using the 'uniform' method."""
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = cv2.LBP(gray, radius, num_points, method=cv2.LBP_UNIFORM)
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        return hist # 26 features
    except:
        return np.zeros(num_points + 2)

def get_hog_features(image: np.ndarray) -> np.ndarray:
    """Calculates Histogram of Oriented Gradients (HOG) features."""
    try:
        resized_image = cv2.resize(image, (64, 128)) 
        hog = cv2.HOGDescriptor()
        h = hog.compute(resized_image)
        return h.flatten() # 3780 features
    except:
        return np.zeros(3780) 

def get_color_histogram(image: np.ndarray) -> np.ndarray:
    """Calculates a 3-channel HSV color histogram."""
    try:
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        histograms = []
        for i in range(3):
            hist = cv2.calcHist([hsv], [i], None, [256], [0, 256])
            hist = cv2.normalize(hist, hist).flatten()
            histograms.append(hist)
        return np.concatenate(histograms) # 768 features
    except:
        return np.zeros(256 * 3)

def extract_traditional_features(images: List[Path]) -> np.ndarray:
    """
    Extracts and combines HOG, LBP, and Color Histogram features for a list of images.
    Total features = 3780 + 26 + 768 = 4574
    """
    features = []
    print("\nExtracting traditional features...")
    for image_path in tqdm(images):
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                continue

            hog_features = get_hog_features(image)
            lbp_features = get_lbp_features(image)
            color_hist = get_color_histogram(image)

            combined_feature = np.concatenate([hog_features, lbp_features, color_hist])
            features.append(combined_feature)
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            
    return np.array(features, dtype=np.float32)

# =============================================================================
# BENCHMARK PLOTTING
# =============================================================================

def plot_benchmark_results(df_benchmark: pd.DataFrame, output_dir: Path):
    """
    Plots and saves benchmark results for time and accuracy.
    """
    print("\nGenerating benchmark plots...")
    
    try:
        # 1. PLOT TIME (FIT & PREDICT)
        print("Generating Time Plot...")
        df_melted = df_benchmark.melt(id_vars=['model', 'library'], 
                                      value_vars=['fit_time', 'pred_time'],
                                      var_name='metric', 
                                      value_name='time_seconds')
        
        df_melted['metric'] = df_melted['metric'].replace({
            'fit_time': 'Training Time (Fit)',
            'pred_time': 'Prediction Time (Predict)'
        })

        g = sns.catplot(
            data=df_melted, 
            x='model', 
            y='time_seconds', 
            hue='library', 
            col='metric', 
            kind='bar', 
            sharey=False, 
            height=6, 
            aspect=1.1,
            legend=False 
        )
        
        g.fig.suptitle('Benchmark: Time Comparison', y=1.05, fontsize=16)
        g.set_axis_labels("Model", "Time (seconds)")
        g.set_titles("{col_name}")

        for ax in g.axes.flat:
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

        g.add_legend(title='Library', bbox_to_anchor=(1.02, 0.5), loc='center left', borderaxespad=0.)
        plt.tight_layout(rect=[0, 0, 0.9, 1]) 

        time_plot_path = output_dir / "benchmark_times.png"
        g.savefig(time_plot_path)
        plt.close('all')
        print(f"Time plot saved at: {time_plot_path}")

        # 2. PLOT ACCURACY
        print("Generating Accuracy Plot...")
        plt.figure(figsize=(14, 8)) 
        
        sns.barplot(data=df_benchmark, x='model', y='accuracy', hue='library')
        
        plt.title('Benchmark: Accuracy Comparison', fontsize=16)
        plt.ylabel('Accuracy')
        plt.xlabel('Model')
        plt.legend(title='Library', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout(rect=[0, 0, 0.9, 1]) 
        
        acc_plot_path = output_dir / "benchmark_accuracy.png"
        plt.savefig(acc_plot_path)
        plt.close()
        print(f"Accuracy plot saved at: {acc_plot_path}")
        
        print("\nPlot generation complete.")

    except Exception as e:
        print(f"An error occurred while plotting: {e}")


# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main function to execute the feature extraction and ML model training pipeline."""
    config = Config()
    
    benchmark_results = []
    
    # Set random seeds for reproducibility
    torch.manual_seed(config.seed)
    np.random.seed(config.seed)
    
    # 1. Prepare data
    images, labels = prepare_data(config)
    
    # Split data (Train/Val/Test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        images, labels, test_size=0.3, random_state=config.seed, stratify=labels
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=config.seed, stratify=y_temp
    )
    
    print(f"\nDataset splits:")
    print(f"Train: {len(X_train)} images")
    print(f"Val:   {len(X_val)} images")
    print(f"Test:  {len(X_test)} images")

    # Combine train and val for a larger training set
    train_val_images = X_train + X_val
    train_val_labels = y_train + y_val
    
    # 2. Extract traditional features
    X_train_features = extract_traditional_features(train_val_images)
    X_test_features = extract_traditional_features(X_test)
    
    y_train_cpu = np.array(train_val_labels, dtype=np.int32)
    y_test_cpu = np.array(y_test, dtype=np.int32)
    
    num_test_images = len(y_test_cpu)
    
    print(f"\nFeature extraction complete.")
    print(f"Train features shape: {X_train_features.shape}")
    print(f"Test features shape:  {X_test_features.shape}")
    
    # ==========================================================================
    # 3. FEATURE SCALING
    # Apply StandardScaler (z-score normalization)
    # ==========================================================================
    print("\nApplying StandardScaler to features...")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_features)
    X_test_scaled = scaler.transform(X_test_features)
    
    print("Feature scaling complete.")
    
    # Save the scaler for inference
    joblib.dump(scaler, config.output_dir / "scaler.joblib")
    print(f"Scaler model saved.")

    # 4. Define models
    
    # SKLEARN (CPU)
    model_name = "KNN_Cosine"
    sklearn_models = {
        model_name: KNeighborsClassifier(
            n_neighbors=5,      # k=5 as requested
            weights='distance', # Use distance-based weighting
            metric='cosine',    # Use Cosine Similarity
            n_jobs=-1           
        ),
    }
    
    # 5. SKLEARN (CPU) Training Loop
    print("\n--- Training and Evaluating SKLEARN (CPU) Models ---")
    for model_name, model in sklearn_models.items():
        print(f"\n--- [SKLEARN] {model_name} (k=5, weights='distance', Scaled, metric='cosine') ---")
        
        # Train
        print("Training on Scaled Data...")
        start_fit = time.time()
        model.fit(X_train_scaled, y_train_cpu)
        fit_time = time.time() - start_fit
        
        # Predict
        print(f"Predicting on {num_test_images} images (Scaled Test data)...")
        start_pred = time.time()
        y_pred = model.predict(X_test_scaled)
        pred_time = time.time() - start_pred
        
        # Evaluate
        accuracy = accuracy_score(y_test_cpu, y_pred)
        time_per_image = pred_time / num_test_images
        
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Fit time: {fit_time:.4f}s")
        print(f"Predict time: {pred_time:.4f}s")
        print(f"Time per Image: {time_per_image:.6f}s")
        
        # Store benchmark results
        benchmark_results.append({
            'library': 'sklearn (CPU)',
            'model': model_name,
            'fit_time': fit_time,
            'pred_time': pred_time,
            'time_per_image': time_per_image,
            'accuracy': accuracy
        })
        
        # Classification Report
        report = classification_report(
            y_test_cpu, 
            y_pred, 
            target_names=config.categories,
            digits=4
        )
        print("\nClassification Report:")
        print(report)
        
        # Save classification report
        report_path = config.output_dir / f"classification_report_SKLEARN_{model_name}.txt"
        with open(report_path, 'w') as f:
            f.write(f"Model: {model_name} (SKLEARN, Scaled, metric=cosine)\n")
            f.write(report)
        print(f"Classification report saved to {report_path}")

        # Save confusion matrix
        cm_path = config.output_dir / f"confusion_matrix_SKLEARN_{model_name}.png"
        plot_confusion_matrix(
            y_true=y_test_cpu,
            y_pred=y_pred,
            class_names=config.categories,
            save_path=cm_path,
            model_name=f"SKLEARN_{model_name}"
        )

        # Save trained model
        model_save_path = config.output_dir / f"SKLEARN_{model_name}.joblib"
        joblib.dump(model, model_save_path)
        print(f"Model saved to {model_save_path}")

    # 6. Finalize, Save, and Plot Benchmark
    print("\n--- Benchmark Results Summary ---")
    if not benchmark_results:
        print("No benchmark results were recorded.")
    else:
        df_benchmark = pd.DataFrame(benchmark_results)
        df_benchmark = df_benchmark.sort_values(by=['model', 'library'])
        
        # Display summary in console
        print(df_benchmark.to_string(columns=[
            'library', 'model', 'fit_time', 'pred_time', 'time_per_image', 'accuracy'
        ]))
        
        # Save to CSV
        csv_path = config.output_dir / "benchmark_results.csv"
        df_benchmark.to_csv(csv_path, index=False)
        print(f"\nBenchmark results saved to {csv_path}")
        
        # Generate plots
        plot_benchmark_results(df_benchmark, config.output_dir)

    print("\nAll models trained and evaluated successfully!")

if __name__ == '__main__':
    main()

CUML/CUPY support is disabled. Running Sklearn models on CPU only.
Loading data paths...
citrus canker: 2600 images
healthy: 2600 images
melanose: 2600 images

Dataset splits:
Train: 5460 images
Val:   1170 images
Test:  1170 images

Extracting traditional features...


100%|██████████| 6630/6630 [05:57<00:00, 18.52it/s]



Extracting traditional features...


100%|██████████| 1170/1170 [01:03<00:00, 18.40it/s]



Feature extraction complete.
Train features shape: (6630, 4574)
Test features shape:  (1170, 4574)

Applying StandardScaler to features...
Feature scaling complete.
Scaler model saved.

--- Training and Evaluating SKLEARN (CPU) Models ---

--- [SKLEARN] KNN_Cosine (k=5, weights='distance', Scaled, metric='cosine') ---
Training on Scaled Data...
Predicting on 1170 images (Scaled Test data)...
Accuracy: 0.9248
Fit time: 0.0131s
Predict time: 0.5684s
Time per Image: 0.000486s

Classification Report:
               precision    recall  f1-score   support

citrus canker     0.9626    0.8590    0.9079       390
      healthy     0.8621    0.9615    0.9091       390
     melanose     0.9612    0.9538    0.9575       390

     accuracy                         0.9248      1170
    macro avg     0.9287    0.9248    0.9248      1170
 weighted avg     0.9287    0.9248    0.9248      1170

Classification report saved to /kaggle/working/classification_report_SKLEARN_KNN_Cosine.txt
Confusion matrix 

/usr/local/lib/python3.11/dist-packages/seaborn/categorical.py:645: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  g_vals = grouped_vals.get_group(g)
/usr/local/lib/python3.11/dist-packages/seaborn/categorical.py:645: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  g_vals = grouped_vals.get_group(g)
/usr/local/lib/python3.11/dist-packages/seaborn/categorical.py:645: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  g_vals = grouped_vals.get_group(g)
/usr/local/lib/python3.11/dist-packages/seaborn/categorical.py:645: FutureWarning: When groupi

Time plot saved at: /kaggle/working/benchmark_times.png
Generating Accuracy Plot...
Accuracy plot saved at: /kaggle/working/benchmark_accuracy.png

Plot generation complete.

All models trained and evaluated successfully!
